#### Importing Library

In [23]:
import logging
import win32com.client
import sqlalchemy
import os
import pandas as pd
import pyodbc as py
import warnings
warnings.filterwarnings('ignore')

#### Config Variable

In [24]:
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=UG-BLR-L262;'
    r'database =AdventureWorksDW2008R2;'
    r'username=ETLdemo;'
    r'password=Devraj@@1;'
    r'Trusted_Connection=yes;'
)

#### Email Block

In [25]:
def sendmail():
    from_addr = 'prakash.raj@ugamsolutions.com'
    to_addr = 'prakash.raj@ugamsolutions.com'
    outlook = win32com.client.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = to_addr
    mail.Subject = 'AB testing log'
    mail.HTMLBody = '<h3>This is HTML Body</h3>'
    mail.Body = 'from:prakash.raj@ugamsolutions.com\nto: prakash.raj@ugamsolutions.com\nEmail contains log files for AB testing'
    mail.Attachments.Add('C:\\Users\\prakash.raj\\demo1.log')
    mail.Send()

#### Logging Config

In [26]:
logging.basicConfig(filename ="demo1.log",filemode='a' ,level=logging.DEBUG, format='%(asctime)s %(levelname)s-%(message)s')

#### Extract function

In [27]:
def extract():
    #Connection being established
    try:
        cnxn=py.connect(conn_str)
        logging.debug("Connection created")
        #query being executed
        sql = """Select CustomerKey,FirstName,LastName,BirthDate,Phone from AdventureWorksDW2008R2.dbo.DimCustomer"""
        results = pd.read_sql(sql, cnxn)
        logging.debug('Query Extracted sucessfully')
        #reading excel file
        df=pd.read_excel("C:/Users/prakash.raj/Downloads/sampledatafoodsales/sampledatafoodsales.xlsx",sheet_name="FoodSales")
        logging.debug('excel extracted sucessfully')
        #reading json file
        df1=pd.read_json("C:/Users/prakash.raj/Downloads/sampledatafoodsales/food.json")
        logging.debug('json extracted sucessfully')
        #transforamtion in process
        df.set_index("CustomerKey", inplace=True)
        df1.set_index("Category", inplace=True)
        df2=results.join(df, on="CustomerKey", how="inner")
        df3=df2.join(df1, on="Category", how="inner")
        df3.set_index("order_id", inplace=True)
        load(df3)
    except exception as e:
        logging.debug("data transformation block" + str(e))
    finally:
        cnxn.close()
   

#### Load Function

In [30]:
def load(df):
    try:
        tbl='sales_detail'
        logging.info("calling load")
        server = 'UG-BLR-L262'
        database = 'ETLtest'
        #creating engine
        sqlcon = sqlalchemy.create_engine('mssql+pyodbc://@' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
        sqlcon.connect()
        logging.debug("engine created")
        df.to_sql(tbl,schema='dbo',con=sqlcon, if_exists ='replace')
        logging.debug("table loaded sucessfully")
        #loading to dashboard
        df.to_excel("C:\\Users\\prakash.raj\\Documents\\Dashboard.xlsx", sheet_name='raw')
        logging.debug("Excel loaded sucessfully")
    except exception as e:
        logging.debug("failed loading block" +str(e))     

#### Main

In [31]:
try:
    extract()
    sendmail()
except exception as e:
    logging.debug("data extraction error" + str(e))
finally:
    logging.shutdown() 